In [ ]:
%reload_ext autoreload

# Imports

import os
import sys
module_path = os.path.abspath(os.path.join('../image_gen_dm')) # or the path to your source code
sys.path.append(str(module_path))

import tensorflow as tf
import tf_keras as tfk
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

tfkl = tfk.layers
tfpl = tfp.layers
tfd = tfp.distributions

TF_ENABLE_ONEDNN_OPTS=0
os.environ['TF_ENABLE_ONEDNN_OPTS'] = str(TF_ENABLE_ONEDNN_OPTS)
os.environ['TG_GPU_ALLOCATOR'] = 'cuda_malloc_async'

import denoising_vae as dvae
import denoising_vae.constants as consts

print('Tensorflow Version:', tf.__version__)
print("Available GPUs: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# Constants

IMAGE_SIZE = 64

ENCODER_HIDDEN_SIZES = [
    [1792, 1152, 448],
    [1792, 1152, 448],
    [1792, 1152, 448],
    [1792, 1152, 448],

    [1792, 1152, 448],
    [1792, 1152, 448],
    [1792, 1152, 448],
    [1792, 1152, 448],

    [1792, 1152, 448],
    [1792, 1152, 448],
    [1792, 1152, 448],
    [1792, 1152, 448],
    
    [1792, 1152, 448],
    [1792, 1152, 448],
    [1792, 1152, 448],
    [1792, 1152, 448],
]

DECODER_HIDDEN_SIZES = [] # Construct from encoder hidden sizes in reverse

for hidden_sizes in ENCODER_HIDDEN_SIZES:
    DECODER_HIDDEN_SIZES.append(hidden_sizes[::-1])


ENCODER_CONV_CONFIGS = [
    [(112, 3, 2), (224, 3, 2), (448, 3, 2), (896, 3, 2), (2048, 3, 2)],
    [(112, 3, 2), (224, 3, 2), (448, 3, 2), (1152, 3, 2), (2048, 3, 2)],
    [(112, 3, 2), (224, 3, 2), (576, 3, 2), (896, 3, 2), (2048, 3, 2)],
    [(112, 3, 2), (224, 3, 2), (576, 3, 2), (1152, 3, 2), (2048, 3, 2)],

    [(112, 3, 2), (288, 3, 2), (448, 3, 2), (896, 3, 2), (2048, 3, 2)],
    [(112, 3, 2), (288, 3, 2), (448, 3, 2), (1152, 3, 2), (2048, 3, 2)],
    [(112, 3, 2), (288, 3, 2), (576, 3, 2), (896, 3, 2), (2048, 3, 2)],
    [(112, 3, 2), (288, 3, 2), (576, 3, 2), (1152, 3, 2), (2048, 3, 2)],

    [(144, 3, 2), (224, 3, 2), (448, 3, 2), (896, 3, 2), (2048, 3, 2)],
    [(144, 3, 2), (224, 3, 2), (448, 3, 2), (1152, 3, 2), (2048, 3, 2)],
    [(144, 3, 2), (224, 3, 2), (576, 3, 2), (896, 3, 2), (2048, 3, 2)],
    [(144, 3, 2), (224, 3, 2), (576, 3, 2), (1152, 3, 2), (2048, 3, 2)],

    [(144, 3, 2), (288, 3, 2), (448, 3, 2), (896, 3, 2), (2048, 3, 2)],
    [(144, 3, 2), (288, 3, 2), (448, 3, 2), (1152, 3, 2), (2048, 3, 2)],
    [(144, 3, 2), (288, 3, 2), (576, 3, 2), (896, 3, 2), (2048, 3, 2)],
    [(144, 3, 2), (288, 3, 2), (576, 3, 2), (1152, 3, 2), (2048, 3, 2)],
]

DECODER_CONV_CONFIGS = [] # Construct from encoder conv configs in reverse

for conv_configs in ENCODER_CONV_CONFIGS:
    DECODER_CONV_CONFIGS.append(conv_configs[::-1])

FLATTENED_SHAPES = [
    (2, 2, 2048),
    (2, 2, 2048),
    (2, 2, 2048),
    (2, 2, 2048),

    (2, 2, 2048),
    (2, 2, 2048),
    (2, 2, 2048),
    (2, 2, 2048),

    (2, 2, 2048),
    (2, 2, 2048),
    (2, 2, 2048),
    (2, 2, 2048),

    (2, 2, 2048),
    (2, 2, 2048),
    (2, 2, 2048),
    (2, 2, 2048),
]

FLATTENED_SIZES = [] # Construct from flattened shapes

for shape in FLATTENED_SHAPES:
    FLATTENED_SIZES.append(shape[0] * shape[1] * shape[2])

LATENT_DIM = 2

MODEL_NAMES = []

for i in range(len(ENCODER_HIDDEN_SIZES)):
    MODEL_NAMES.append(f'cvae{i+1+28}')

LEARNING_RATE = 1e-4

VAL_PERCENTAGE = 0.1

# Models to run

MODEL_START = 6 # Rasmus 0-7, Viktor 8-15
MODEL_COUNT = 1

In [ ]:
# Dataset loading

train_ds, val_ds = dvae.utils.load_datasets(val_percentage=VAL_PERCENTAGE)

In [ ]:
# Model class

class Sampling(tfkl.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

class CVAE(tfk.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = tfk.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = tfk.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = tfk.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
    
    def train_step(self, data):
        with tf.GradientTape() as tape:
            input_data, target_data = data
            noise_levels = tf.random.uniform(shape=(consts.BATCH_SIZE,), minval=0, maxval=consts.NOISE_LEVELS - 1, dtype=tf.int32)

            def apply_noise_to_image(image, noise_level):
                return dvae.utils.apply_noise(image, noise_level)
            
            input_data_noisy = tf.map_fn(
                lambda x: apply_noise_to_image(x[0], x[1]), 
                (input_data, noise_levels), dtype=tf.float32)

            z_mean, z_log_var, z = self.encoder(input_data_noisy)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    tf.losses.binary_crossentropy(target_data, reconstruction), 
                    axis=(1,2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }
    
    def call(self, input_data, training=False):
        _, _, z = self.encoder(input_data)
        reconstructed_image = self.decoder(z)

        return reconstructed_image


In [ ]:
# Encoder

models = []

for i in range(MODEL_START, MODEL_START + MODEL_COUNT):
    print(f'Model {i+1+28}')

    image_input = tfkl.Input(shape=consts.INPUT_SHAPE, name='image_input')
    
    x = image_input
    for filters, kernel_size, strides in ENCODER_CONV_CONFIGS[i]:
        x = tfkl.Conv2D(filters=filters, kernel_size=(kernel_size, kernel_size), strides=(strides, strides), activation='relu', padding='same')(x)

    flattened_image = tfkl.Flatten()(x)

    x = flattened_image
    for layer_size in ENCODER_HIDDEN_SIZES[i]:
        x = tfkl.Dense(layer_size, 
                    activation='relu',
                    kernel_initializer='glorot_uniform')(x)
        
    z_mean = tfkl.Dense(LATENT_DIM, name='z_mean', kernel_initializer='glorot_uniform')(x)
    z_log_var = tfkl.Dense(LATENT_DIM, name='z_log_var', kernel_initializer='glorot_uniform')(x)

    z = Sampling()([z_mean, z_log_var])

    encoder = tfk.Model(inputs=image_input, outputs=[z_mean, z_log_var, z], name='encoder')
    encoder.summary()

    # Decoder

    latent_inputs = tfkl.Input(shape=(LATENT_DIM,))

    y = latent_inputs
    for layer_size in DECODER_HIDDEN_SIZES[i]:
        y = tfkl.Dense(layer_size, 
                       activation='relu',
                       kernel_initializer='glorot_uniform')(y)
        
    reconstructed_flattened_image = tfkl.Dense(FLATTENED_SIZES[i], kernel_initializer='glorot_uniform', activation='relu')(y)
    
    y = tfkl.Reshape(FLATTENED_SHAPES[i])(reconstructed_flattened_image)
   
    j = 0
    for filters, kernel_size, strides in DECODER_CONV_CONFIGS[i]:
        if j == len(DECODER_CONV_CONFIGS[i]) - 1:
            y = tfkl.Conv2DTranspose(filters=3, kernel_size=(kernel_size, kernel_size), strides=(strides, strides), activation='sigmoid', padding='same')(y)
        else:
            y = tfkl.Conv2DTranspose(filters=DECODER_CONV_CONFIGS[i][j + 1][0], kernel_size=(kernel_size, kernel_size), strides=(strides, strides), activation='relu', padding='same')(y)
        j += 1

    decoder = tfk.Model(inputs=latent_inputs, outputs=y, name='decoder')
    decoder.summary()

    # Model
        
    model = CVAE(encoder, decoder)
    model.compile(optimizer=tfk.optimizers.Adam(learning_rate=LEARNING_RATE))

    models.append(model)



In [ ]:
# Run model

epochs = 128

i = MODEL_START
for model in models:

    history = dvae.run_model(model=model, train_ds=train_ds, val_ds=val_ds, epochs=epochs, name=MODEL_NAMES[i], plot=False)
    
    pd.DataFrame(history.history).to_csv(f'res/histories/{MODEL_NAMES[i]}.csv')

    dvae.val_model(model)

    i += 1
